LSTM model implementation

In [4]:
 from keras import layers
 from keras import models
 model = models.Sequential()
 model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
 model.add(layers.MaxPooling2D((2, 2)))
 model.add(layers.Conv2D(64, (3, 3), activation='relu'))
 model.add(layers.MaxPooling2D((2, 2)))
 model.add(layers.Conv2D(64, (3, 3), activation='relu'))

C:\Users\juankroos\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [3]:
# Chemin vers les données extraites
DATA_PATH = os.path.join(r'E:\Gesture-Recognition-using-3D-CNN\MP_Data1')

# Récupérer automatiquement les actions (noms des dossiers dans MP_Data)
actions = np.array([d for d in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, d))])


In [4]:
actions

array(['hello', 'help', 'learn', 'like', 'month', 'need'], dtype='<U5')

In [5]:
# Vérifier si des actions ont été trouvées
if len(actions) == 0:
    print(f"Erreur : Aucun dossier d'action trouvé dans {DATA_PATH}.")
    exit()

print(f"Actions détectées : {actions}")

# Nombre de frames par vidéo
sequence_length = 30

# Créer un dictionnaire pour mapper les actions à des indices
label_map = {label: num for num, label in enumerate(actions)}


Actions détectées : ['hello' 'help' 'learn' 'like' 'month' 'need']


In [6]:
# Charger les données
sequences, labels = [], []
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    if not os.path.exists(action_path):
        print(f"Erreur : Le dossier {action_path} n'existe pas.")
        continue
    
    # Parcourir les séquences (dossiers numérotés)
    sequence_dirs = [d for d in os.listdir(action_path) if os.path.isdir(os.path.join(action_path, d))]
    for sequence in sequence_dirs:
        try:
            sequence = int(sequence)  # Convertir en entier
            window = []
            for frame_num in range(sequence_length):
                npy_path = os.path.join(action_path, str(sequence), f"{frame_num}.npy")
                if not os.path.exists(npy_path):
                    print(f"Fichier manquant : {npy_path}")
                    break
                res = np.load(npy_path)
                window.append(res)
            else:  # S'exécute si la boucle n'est pas interrompue
                sequences.append(window)
                labels.append(label_map[action])
        except ValueError:
            print(f"Nom de dossier invalide (non numérique) : {sequence}")
            continue


Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\month\13\25.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\month\14\25.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\month\15\25.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\month\16\25.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\month\17\25.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\month\18\25.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\need\13\29.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\need\14\29.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\need\15\29.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\need\16\29.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\need\17\29.npy
Fichier manquant : E:\Gesture-Recognition-using-3D-CNN\MP_Data1\need\18\29.npy


In [7]:
# Convertir en tableaux NumPy
X = np.array(sequences)
y = to_categorical(labels).astype(int)
# Vérifier les formes des données
print(f"Forme de X : {X.shape}")
print(f"Forme de y : {y.shape}")

Forme de X : (185, 30, 258)
Forme de y : (185, 6)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [11]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [12]:
from tensorflow.keras.layers import Bidirectional
model = Sequential()

model.add(Bidirectional(LSTM(64, return_sequences=True, activation='relu')))
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length, X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

C:\Users\juankroos\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Compiler le modèle
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [15]:
# Afficher le résumé du modèle
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 30, 64)              │          82,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 30, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 237,350 (927.15 KB)

 Trainable params: 237,350 (927.15 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Entraîner le modèle
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), callbacks=[tb_callback])

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 268ms/step - categorical_accuracy: 0.2573 - loss: 1.7233 - val_categorical_accuracy: 0.3000 - val_loss: 1.4513
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - categorical_accuracy: 0.4798 - loss: 1.3302 - val_categorical_accuracy: 0.5000 - val_loss: 1.5727
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - categorical_accuracy: 0.5307 - loss: 1.1885 - val_categorical_accuracy: 0.8000 - val_loss: 0.8173
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - categorical_accuracy: 0.6701 - loss: 0.9184 - val_categorical_accuracy: 0.5000 - val_loss: 1.0405
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - categorical_accuracy: 0.5660 - loss: 0.9526 - val_categorical_accuracy: 0.7000 - val_loss: 0.6507
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - categorical_accuracy: 0.5984 - loss: 1.2959 - val_categorical_accuracy: 1.0000 - val_loss: 0.6631
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - categorical_accuracy: 0.5562 - loss: 0.9623 - 

In [17]:
# Évaluer le modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Perte sur l'ensemble de test : {loss:.4f}")
print(f"Précision sur l'ensemble de test : {accuracy:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - categorical_accuracy: 0.8000 - loss: 0.3165
Perte sur l'ensemble de test : 0.3165
Précision sur l'ensemble de test : 0.8000


In [18]:
# Faire des prédictions
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step


In [22]:
# Vérifier quelques prédictions
for i in range(min(5, len(X_test))):
    predicted_action = actions[np.argmax(res[i])]
    true_action = actions[np.argmax(y_test[i])]
    print(f"Échantillon {i+1} : Prédit = {predicted_action}, Vrai = {true_action}")


Échantillon 1 : Prédit = hello, Vrai = hello
Échantillon 2 : Prédit = hello, Vrai = hello
Échantillon 3 : Prédit = month, Vrai = month
Échantillon 4 : Prédit = learn, Vrai = learn
Échantillon 5 : Prédit = month, Vrai = month


In [23]:
# Sauvegarder le modèle
model.save('action1.h5')
print("Modèle sauvegardé sous 'action1.h5'.")

Modèle sauvegardé sous 'action1.h5'.
